<a href="https://colab.research.google.com/github/sergiolms/smartbite/blob/main/google-collab/Smartbite_Showcase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SmartBite 🍏
The AI tool that makes you eat wiser.

## 🥫 Dataset used
- Dataset Food101 extracted from [Kaggle](https://www.kaggle.com/datasets/dansbecker/food-101).
  - This dataset contains 101 folders of food & dishes with a thousand pictures each.

- For the food information, due to time limitations we initially used ChatGPT to obtain the list of ingredients & nutritional information based on a recommended portion in grams.
  - As a follow up task, we intend to use the [Open Food Facts database](https://world.openfoodfacts.org/data), a ~10GB database of food and ingredients with their nutritional information and healthy score. This will give us a more precise and adjusted result of the nutritional information of the dish.

## 🍽 Project Setup
Install dependencies, make imports and mount Google Drive

In [ ]:
# Check your tensorflow version. If it's 2.15, run the following fragment
import tensorflow as tf
print(tf.__version__)

In [ ]:
# Uninstall Tensorflow v2.15 and install v2.16.1
!pip uninstall tensorflow -y
!pip install tensorflow==2.16.1

In [ ]:
# Run this if it's your first time. The system needs to install gradio to generate the interface.
!pip install gradio

In [3]:
import os
import re
import json
import random
import tensorflow as tf
import gradio as gr
import numpy as np
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image

from google.colab import drive

In [9]:
# Mount the drive
drive.mount('/content/drive')
# Here we load our project folder, that has the following structure:
# - root folder ("smartbite_project")
# |-- food_info // Contains the nutritional information JSON file
# |-- images // Contains all images
# |-- meta // Contains metadata files, such as training & test data and classes
# |-- model_trained_3class.keras // This is the model trained
# Change this path to the location of this project in your Drive
base_folder = '/content/drive/MyDrive/smartbite_project/'
model_name = 'model_trained_3class.hdf5'

Mounted at /content/drive


## ☕️ Function declaration
In this section we will declare some methods that will help us in the process

In [6]:
# Read a JSON file and return its contents
def get_info_json(file):
  with open(file, "r", encoding="UTF-8") as f:
    info_food = json.load(f)
  return info_food

# Get the category name from the numeric representation returned by the model
def get_category(label_numeric, labels_strings_food):
  if label_numeric >= 0 and label_numeric < len(labels_strings_food):
    return labels_strings_food[label_numeric]

# Given an image, process it and make a prediction with the model
def predict_class(model, img):
  # Process image
  img = image.load_img(img, target_size=(299, 299))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img /= 255.

  # Generate prediction
  return np.argmax(model.predict(img))

def display_nutritional_information(imagen):
    prediction = predict_class(model, imagen)
    guessed_food = get_category(int(prediction), labels_strings_food)
    data = info_food[guessed_food]
    return "\n".join(f"{key}: {value}" for key, value in data.items())

## 🧑🏻‍🍳 Load the model & categories

In [10]:
# Load the trained model
model = models.load_model(os.path.join(base_folder, model_name), compile=False)

In [11]:
# Get the nutritional information JSON file
info_food = get_info_json(os.path.join(base_folder, "food_info/food_info.json"))

# Store keys in a sorted array
labels_strings_food = []
for key in info_food:
    labels_strings_food.append(key)
labels_strings_food.sort()

## 🍪 Demo time! 💥

In [ ]:
demo = gr.Interface(
    fn=display_nutritional_information, # On image sent, run this fn
    inputs=gr.Image(type="filepath", label="Take a photo or upload a picture of your dish"),
    outputs=["text"],
    title="SmartBite",
    description="Upload a picture of your dish to see its nutritional information."
)

demo.launch()